In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Recommender Systems').getOrCreate()
spark

In [3]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
movies = spark.read.csv('./data/movielens_ratings.csv', 
                        header= True, 
                        inferSchema=True) 

In [5]:
movies.show(5)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
+-------+------+------+
only showing top 5 rows



In [6]:
movies.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [8]:
xTrain, xTest = movies.randomSplit((0.8, 0.2), seed= 42)

In [9]:
als = ALS(maxIter= 5, 
          regParam= 0.01, 
          userCol= 'userId', 
          itemCol= 'movieId', 
          ratingCol= 'rating')

In [10]:
model = als.fit(xTrain)

In [11]:
predictions = model.transform(xTest)

In [12]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    27|  0.8712373|
|     31|   1.0|     5|-0.42556602|
|     31|   1.0|    19| 0.92135257|
|     31|   3.0|    14|  2.9411464|
|     31|   1.0|     0|  0.8908188|
|     85|   3.0|     6|  4.3408704|
|     85|   4.0|     7|  4.0961556|
|     53|   1.0|    12|  2.6252193|
|     78|   1.0|    28|  1.0353686|
|     34|   1.0|    16|   4.521089|
|     34|   1.0|    15|  1.4414614|
|     34|   1.0|     0|-0.85931534|
|     28|   3.0|     1|  2.6998339|
|     28|   1.0|     5|  3.1425374|
|     28|   1.0|     2|   3.418564|
|     76|   1.0|     1|  0.7603221|
|     76|   1.0|    19|  0.5588336|
|     76|   3.0|     7|  3.0901005|
|     76|   1.0|    25|  1.2033222|
|     76|   1.0|     2|   3.848323|
+-------+------+------+-----------+
only showing top 20 rows



In [13]:
evaluator = RegressionEvaluator(metricName= 'rmse', 
                                labelCol= 'rating',
                                predictionCol= 'prediction')

In [14]:
rmse = evaluator.evaluate(predictions)

In [15]:
print(f'RMSE : {rmse}')

RMSE : 1.6955294020218656


In [16]:
single_user = xTest.filter(xTest['userId'] == 11).select(['movieId', 'userId'])

In [17]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     11|    11|
|     32|    11|
|     59|    11|
|     62|    11|
|     66|    11|
|     67|    11|
+-------+------+



In [18]:
recommandations = model.transform(single_user)

In [ ]:
recommandations.orderBy('prediction', ascending = False).show()